In [ ]:
import cv2
import numpy as np
import glob

# Chessboard settings (adjust based on your board)
chessboard_size = (10, 7)  # 11x8 chessboard dimensions
square_size = 25  # in mm (for real-world scale, optional)

# Prepare object points (3D)
objp = np.zeros((np.prod(chessboard_size), 3), np.float32)
objp[:, :2] = np.mgrid[0:chessboard_size[0], 0:chessboard_size[1]].T.reshape(-1, 2) * square_size

objpoints = []  # 3D world points
imgpoints = []  # 2D image points

# Load calibration images
images = glob.glob("Calibration_Images/*.jpg")  # Only JPG  # Match all files # Path to saved images

for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Find chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, chessboard_size, None)

    if ret:
        objpoints.append(objp)
        imgpoints.append(corners)

        # Draw and show corners
        cv2.drawChessboardCorners(img, chessboard_size, corners, ret)
        cv2.imshow('Corners', img)
        cv2.waitKey(500)

cv2.destroyAllWindows()

# Camera Calibration
ret, camera_matrix, dist_coeffs, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)




